# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
print(f'Количество полных дубликатов: {data.duplicated().sum()}')

Количество полных дубликатов: 153


In [5]:
data.shape

(5000, 5)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


1. Таблица содержит 5000 наблюдений по 5 признаков в каждом
2. Пропуски в данных отсутствуют
3. Совокупность имеющихся признаков не позволяет говорить о наличии/отсутствии дубликатов со всей полнотой уверенности несмотря на 153 идентинчых строки

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

**Задача**:

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится

**Обоснование:** 


Так как предсказания вычисляются по уже известной нам формуле:

$$
w = (X^T X)^{-1} X^T y
$$


Формула для расчёта предсказаний при умножении матрицы признаков на обратимую матрицу будут выглядеть следующим образом (где, P - случайная обратимая матрица):

$$
w_1 = ((XP)^TXP)^{-1}(XP)^Ty
$$

Эту же формулу можно записать в другом виде:

$$
w_1 = (X^TP^TXP)^{-1}X^TP^Ty
$$

Далее раскрываем скобки и немного перегруппируем множители:

$$
w_1 = P^{-1}(X^TX)(P^T)^{-1}P^TX^Ty
$$

Так как $A^{-1}A=E$, то и $(P^T)^{-1}P^T = E$:

$$
w_1 = P^{-1}(X^TX)^{-1}EX^Ty
$$

Так как $AE=EA=A$, то:

$$
w_1 = P^{-1}(X^TX)^{-1}X^Ty
$$

Так как $w=(X^TX)^{-1}X^Ty$, то формулу можно записать следующим образом:

$$
w1=P^{-1}w
$$

Получить вектор предсказаний для исходной матрицы $Х$ мы можем по формуле $a=Xw$. Из этого следует, что вектор предсказаний $a1$ для видоизмененной матрицы $X1$ может быть получен по формуле $a1=X1w1$, где $X1=XP$. Следовательно: 

$$
a1=XPP^{-1}w
$$

А так как $A^{-1}A=E$, то и $PP^{-1} = E$:

$$
a1=XEw
$$

Или:

$$
a1=Xw
$$

Из чего следует: 

$$
a1=Xw=a
$$

**Вывод:** 

При умножении матрицы признаков на случайную обратимую матрицу качество предсказаний не изменится.

## Алгоритм преобразования

**Алгоритм**

Зашифруем признаки при помощи умножения матрицы признаков MN на случайную обратимую матрицу NN. Таким образом матрица, на которую мы умножим исходные данные, станет своеобразным ключом шифрования, а она же инвертированная позволит дешифровать данные. 

Ранее теоретически было обосновано, что такой алгоритм не ухудшит качество прогнозирования при помощи линейной регрессии, позволив защитить персональные данные клиентов. 

**Шаг 1** - Формируем случайную матрицу. Матрица должна быть квадратной со стороной, равной количеству признаков на основе которых мы производим предсказание.

In [7]:
#генерируем матрицу
random_matrix = np.random.normal(size=(4,4))
random_matrix

array([[-0.1158487 , -0.21041205, -2.449859  , -0.72837116],
       [ 1.88700748,  0.56964131, -0.35911481,  1.26729595],
       [-0.29689521,  0.03663712,  0.14743596,  1.12063037],
       [ 2.92854064,  0.03015761,  0.65571972,  1.57083096]])

**Шаг 2** - Проводим проверку полученной матрицы на обратимость

In [8]:
np.linalg.inv(random_matrix)

array([[ 0.04794785,  0.02835839, -0.3997853 ,  0.2845608 ],
       [-0.58478746,  1.64067893, -0.63145697, -1.14432314],
       [-0.38473857, -0.13377353, -0.17360622,  0.05337721],
       [ 0.08243995, -0.02852613,  0.82992238,  0.1057795 ]])

**Шаг 3** - Производим подготовку данных

In [9]:
#разделим данные на признаки и таргет
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

In [10]:
#разделим данные на тренировочную и тестовую выборки
train_features, test_features, train_target, test_target = train_test_split(
        features,
        target,
        train_size=0.75,
        random_state=12345
    )

In [11]:
#произведем масштабирование признаков
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

**Шаг 4** - Проверка

In [12]:
#создаем класс, который строит прогноз с использованием линейной регрессии
class Linear_Regression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T @ X) @ X.T @ y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [13]:
#обучаем регрессию и сохраняем вектор предикта на чистых данных
model = Linear_Regression()
model.fit(train_features, train_target)
initial_pred = model.predict(test_features)

In [14]:
#проведем преобразование путем умножения на ранее сформированную случайную матрицу
train_features_modified = train_features @ random_matrix
test_features_modified = test_features @ random_matrix

In [15]:
#повторно обучаем модель и сохраняем новый вектор предикта, но уже на измененных данных
model.fit(train_features_modified, train_target)
modified_pred = model.predict(test_features_modified)

In [16]:
print(f'Суммарная разность значений для двух полученных векторов предсказаний: {(initial_pred - modified_pred).sum()}')

Суммарная разность значений для двух полученных векторов предсказаний: 1.6528445279107018e-14


Мы наблюдаем исчезающе малую суммарную разность векторов предсказаний, что может быт обусловлено погрешностью при вычислениях. Проведем еще одну проверку, округлив значения:

In [17]:
predictions = pd.DataFrame(initial_pred, modified_pred).reset_index()
predictions.columns=['initial', 'modified']
predictions = predictions.round(12)
predictions['is_equal?'] = predictions['initial'] == predictions['modified']
predictions

,initial,modified,is_equal?
0,0.174948,0.174948,True
1,0.805235,0.805235,True
2,0.455993,0.455993,True
3,-0.236681,-0.236681,True
4,0.460185,0.460185,True
...,...,...,...
1245,0.866582,0.866582,True
1246,0.195173,0.195173,True
1247,0.312992,0.312992,True
1248,0.349261,0.349261,True


In [18]:
print(f'Совпадает {predictions["is_equal?"].sum()} значений из {len(predictions)} ')  

Совпадает 1250 значений из 1250 


Качество предсказаний не изменилось.

**Шаг 5** - Восстановление исходных данных

In [19]:
#восстановим данные путем умножения преобразованной матрицы на обратную матрицу
decrypted_train_features = train_features_modified @ np.linalg.inv(random_matrix)
decrypted_test_features = test_features_modified @ np.linalg.inv(random_matrix)

In [20]:
print(f'Суммарная разность значений для двух полученных векторов предсказаний: {(test_features - decrypted_test_features).sum()}')

Суммарная разность значений для двух полученных векторов предсказаний: -8.508862017753138e-14


In [28]:
#сравним фреймы с исходными и восстановленными фичами
(pd.DataFrame(decrypted_test_features).round(12) == pd.DataFrame(test_features).round(12)).sum()

0    1250
1    1250
2    1249
3    1250
dtype: int64

Различий в восстановленных и исходных признаках нет. 

## Проверка алгоритма

Повторим процедуру сравнения предсказаний, но с использованием алгоритма линейной регрессии из библиотеки sklearn и метрик MSE/R2. 

In [22]:
model = LinearRegression()
model.fit(train_features, train_target)
initial_pred = model.predict(test_features)

In [23]:
#повторно обучаем модель и сохраняем новый вектор предикта, но уже на измененных данных
model.fit(train_features_modified, train_target)
modified_pred = model.predict(test_features_modified)

In [24]:
#считаем метрики
print(f'R2 предсказаний на исходных данных - {r2_score(test_target, initial_pred)}')
print(f'MSE предсказаний на исходных данных - {mean_squared_error(test_target, initial_pred)}')
print()
print(f'R2 предсказаний на модифицированных данных - {r2_score(test_target, modified_pred)}')
print(f'MSE предсказаний на исходных данных - {mean_squared_error(test_target, modified_pred)}')

R2 предсказаний на исходных данных - 0.4352275712702668
MSE предсказаний на исходных данных - 0.11660517472525563

R2 предсказаний на модифицированных данных - 0.4352275712702668
MSE предсказаний на исходных данных - 0.11660517472525563


Мы видим, что метрики качества модели практически идентичный (отличие в 16-17 знаке после точки скорее всего являются погрешностями вычислений).

## Вывод

1. Установлено, что преобразование данных с целью защиты путем перемножения их на обратимую матрицу не ухудшает качество прогнозирования с использованием алгоритма линейной регрессии
2. Утверждение проверено эмпирически путем сравнения метрик качества прогнозирования и сравнения непосредственно значений